In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()

### GPT Model

In [4]:
system_message_01 = "Youre a assitant that answer for a given question in markdown format in more formal way"
system_message_02 = "Youre a assitant that answer for a given question in markdown format in more fancy way"

def gpt_user_prompt(question):
    user_prompt = f"This is the question: {question}"
    return user_prompt

In [5]:
def gpt_model_formal_answering(question):

    response = openai.chat.completions.create(
        model = 'gpt-4.1-mini',
        messages = [
            {
                "role": "system", "content": system_message_01,
                "role": "user", "content": gpt_user_prompt(question),
            }
        ],
        stream = True 
    )
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

In [6]:
def gpt_model_fancy_answering(question):

    response = openai.chat.completions.create(
        model = 'gpt-4.1-mini',
        messages = [
            {
                "role": "system", "content": system_message_02,
                "role": "user", "content": gpt_user_prompt(question),
            }
        ],
        stream = True 
    )
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

### Claude Model

In [7]:
def claude_model_formal_answering(question):

    result = claude.messages.stream(
        model = "claude-sonnet-4-20250514",
        max_tokens = 1000,
        system = system_message_01,
        messages = [
            {
                "role": "user", "content": gpt_user_prompt(question)
            }
        ],
        temperature = 0.9
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [8]:
def claude_model_fancy_answering(question):

    result = claude.messages.stream(
        model = "claude-sonnet-4-20250514",
        max_tokens = 1000,
        system = system_message_02,
        messages = [
            {
                "role": "user", "content": gpt_user_prompt(question)
            }
        ],
        temperature = 0.9
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

### Gradio Dashboard

In [9]:
def final_model_flow(question, model, answer_type):
    if (model == "GPT" and answer_type== "Formal"):
        response = gpt_model_formal_answering(question)
    elif (model == "GPT" and answer_type== "Fancy"):
        response = gpt_model_fancy_answering(question)
    elif (model == "Claude" and answer_type== "Formal"):
        response = claude_model_formal_answering(question)
    else:
        response = claude_model_fancy_answering(question)
    yield from response

In [10]:
view = gr.Interface(
    fn=final_model_flow,
    inputs=[
        gr.Textbox(label="Your message:"), 
        gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT"), 
        gr.Dropdown(["Formal", "Fancy"], label="Select Answer Type", value="Formal")
    ],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
